In [ ]:
import random
import csv

from shop_crawler import *
import common_actors

### Prepare different urls to analyze

In [ ]:
# All urls
all_urls = []
with open('../resources/pvio_vio_us_ca_uk_sample1.csv', 'r') as f:
    rows = csv.reader(f)
    for row in rows:
        url = row[0]
        if url:
            all_urls.append(url)

# Random sample urls
random.seed(0)
sample_urls = random.sample(all_urls, 20)

# Some good urls to analyze by hands
good_urls = [
    'curlebotanicals.com',
    'theglamourshop.com',
    'vape-fuel.com',
    'firstfitness.com',
    'sandlakedermatology.com',
    'getwaave.com',
    'dixieems.com',
    'jonessurgical.com',
    'srandd.com',
    'ambarygardens.com',
    'anabolicwarfare.com'
]

### Mock user and payment information

In [ ]:
user_info = UserInfo(
    first_name = 'John',
    last_name = 'Smith',
    country = 'United States',
    home = 34,
    street = 'Ocean drive',
    city = 'Miami',
    zip = '33125',
    state = 'FLorida',
    
    phone = '1231232',
    email = 'john@service.com'
)

billing_info = PaymentInfo(
    card_number = '1413232312312321',
    expire_date_year = 2020,
    expire_date_month = 12,
    cvc = '123'
)

### Start crawling for every url

In [ ]:
crawler = ShopCrawler(user_info, billing_info, '/home/aleksei/dist/selenium/chromedriver', headless=True)
common_actors.add_crawler_extensions(crawler)

for url in sample_urls:
    print()
    print('\n\n{}'.format(url))
    crawler.crawl(url)
